In [72]:
#import modules to establish 

from sqlalchemy import create_engine
import pandas as pd
import pyodbc
import json 


#Create engine for interface 
# mssql+pyodbc sting from syntax 
#VICKY Server name for Windows Authentication
#VDI database name 
#driver is for PyOdbc 
#trusted connection


engine = "mssql+pyodbc://VICKY/VDI?driver=ODBC Driver 17 for SQL Server?trusted_connection=yes"
    

In [53]:
# Testing the connection

df = pd.read_sql_table("test",engine)
df

,accountid,name
0,12233,Bill
1,4553,Gifi


In [66]:
# Importing data in JSON format

file_name = r'C:\Users\sanjay\Desktop\VDI\sampledata.json'
with open(file_name, 'r') as f:
    document =  json.loads(f.read())
print(document)

{'@odata.context': '[Organization URI]/api/data/v9.1/$metadata#accounts(name)', 'value': [{'@odata.etag': 'W/"501097"', 'name': 'Fourth Coffee (sample)', 'accountid': '89390c24-9c72-e511-80d4-00155d2a68d1'}, {'@odata.etag': 'W/"501098"', 'name': 'Litware, Inc. (sample)', 'accountid': '8b390c24-9c72-e511-80d4-00155d2a68d1'}, {'@odata.etag': 'W/"501099"', 'name': 'Adventure Works (sample)', 'accountid': '8d390c24-9c72-e511-80d4-00155d2a68d1'}]}


In [71]:
# Processing data into tables

columns = [dct['name'] for dct in document['value']]
df = pd.DataFrame(document['value'])
df

,@odata.etag,accountid,name
0,"W/""501097""",89390c24-9c72-e511-80d4-00155d2a68d1,Fourth Coffee (sample)
1,"W/""501098""",8b390c24-9c72-e511-80d4-00155d2a68d1,"Litware, Inc. (sample)"
2,"W/""501099""",8d390c24-9c72-e511-80d4-00155d2a68d1,Adventure Works (sample)


In [79]:
#Create a table in the database to hold the values

from sqlalchemy import MetaData, Table, Column, Integer, String

engine =create_engine("mssql+pyodbc://VICKY/VDI?driver=ODBC Driver 17 for SQL Server?trusted_connection=yes")

meta = MetaData()

Vdi_data = Table(
   'Vdi_data', meta, 
   Column('@odata.etag', String,), 
   Column('accountid', String), 
   Column('name', String), 
)
meta.create_all(engine)

In [81]:
df.to_sql(
    name='Vdi_data',
    con=engine,
    index=False,
    if_exists='append'
)

In [84]:
df2 = pd.read_sql_table("Vdi_data",engine)
df2

,@odata.etag,accountid,name
0,"W/""501097""",89390c24-9c72-e511-80d4-00155d2a68d1,Fourth Coffee (sample)
1,"W/""501098""",8b390c24-9c72-e511-80d4-00155d2a68d1,"Litware, Inc. (sample)"
2,"W/""501099""",8d390c24-9c72-e511-80d4-00155d2a68d1,Adventure Works (sample)
